In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    dim_hidden: int = 128
    num_inds: int = 64
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.proj = nn.Linear(config.dim_context + config.dim_input, config.dim_hidden)
        self.enc = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        x = x.expand(-1, context.shape[1], -1)      # [batch_size, context_size, dim_input]
        context = torch.cat([context, x], dim=-1)   # [batch_size, context_size, dim_context + dim_input]
        context = self.proj(context)                # [batch_size, context_size, dim_hidden]
        y = self.enc(context)           # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(y, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': y}

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [21]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [22]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_12_27_enc_only'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_12_06_no_enc-2024-12-27-17-16-53


In [23]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.02,   "lambda": 0.0,  'learning_rate': 5e-5},
    {"epochs": 2, "epsilon": 0.0075, "lambda": 0.15, 'learning_rate': 4e-5},
    {"epochs": 2, "epsilon": 0.004,  "lambda": 0.3,  'learning_rate': 3e-5},
    {"epochs": 2, "epsilon": 0.002,  "lambda": 0.4,  'learning_rate': 2e-5},
    {"epochs": 2, "epsilon": 0.0,    "lambda": 0.5,  'learning_rate': 1e-5}
]

In [ ]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=100, context_size=200)
train_dataset.close()
val_dataset.close()

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.0087, 'grad_norm': 0.0, 'learning_rate': 4.997222222222223e-05, 'epoch': 0.0}
{'loss': 0.0049, 'grad_norm': 0.10689723491668701, 'learning_rate': 4.994444444444445e-05, 'epoch': 0.0}
{'loss': 0.0061, 'grad_norm': 0.035397496074438095, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.0}
{'loss': 0.0019, 'grad_norm': 0.07025542110204697, 'learning_rate': 4.9888888888888894e-05, 'epoch': 0.0}
{'loss': 0.0029, 'grad_norm': 0.03543964773416519, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.01}
{'loss': 0.0023, 'grad_norm': 0.0, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 0.0024, 'grad_norm': 0.03504539281129837, 'learning_rate': 4.9805555555555554e-05, 'epoch': 0.01}
{'loss': 0.0049, 'grad_norm': 0.0355084128677845, 'learning_rate': 4.977777777777778e-05, 'epoch': 0.01}
{'loss': 0.0022, 'grad_norm': 0.03549712151288986, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 0.0033, 'grad_norm': 0.10482480376958847, 'learning_rate': 4.972222222222223e-05, 

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 0.0012973910197615623, 'eval_runtime': 2.2861, 'eval_samples_per_second': 1749.689, 'eval_steps_per_second': 43.742, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.12348215281963348, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.05057787150144577, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0012, 'grad_norm': 0.09940856695175171, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0009, 'grad_norm': 0.276979923248291, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0008, 'grad_norm': 0.14507918059825897, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.001, 'grad_norm': 0.11468509584665298, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0006, 'grad_norm': 0.1425214558839798, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0015, 'grad_norm': 0.052421316504478455, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss':

  0%|          | 0/100 [00:00<?, ?it/s]

...

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.0042, 'grad_norm': 0.4883591830730438, 'learning_rate': 4.997222222222223e-05, 'epoch': 0.0}
{'loss': 0.0054, 'grad_norm': 0.563546895980835, 'learning_rate': 4.994444444444445e-05, 'epoch': 0.0}
{'loss': 0.005, 'grad_norm': 1.177344560623169, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.0}
{'loss': 0.0034, 'grad_norm': 0.6284283399581909, 'learning_rate': 4.9888888888888894e-05, 'epoch': 0.0}
{'loss': 0.0026, 'grad_norm': 0.38085663318634033, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.01}
{'loss': 0.0017, 'grad_norm': 0.32772114872932434, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 0.0025, 'grad_norm': 0.3030850291252136, 'learning_rate': 4.9805555555555554e-05, 'epoch': 0.01}
{'loss': 0.0033, 'grad_norm': 0.23356223106384277, 'learning_rate': 4.977777777777778e-05, 'epoch': 0.01}
{'loss': 0.0017, 'grad_norm': 0.08875414729118347, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 0.0016, 'grad_norm': 0.35776287317276, 'learning_rate': 4.

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 0.0016550632426515222, 'eval_runtime': 2.2621, 'eval_samples_per_second': 1768.288, 'eval_steps_per_second': 44.207, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.14280039072036743, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.0015, 'grad_norm': 0.1436377614736557, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0016, 'grad_norm': 0.1743851900100708, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0007, 'grad_norm': 0.19771862030029297, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0008, 'grad_norm': 0.03780263289809227, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.001, 'grad_norm': 0.16972163319587708, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0005, 'grad_norm': 0.06666935980319977, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0019, 'grad_norm': 0.1279045045375824, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss': 

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)